In [1]:
'''
Copyright (C) 2022 Francesco Paparella, Pedro Velasquez

This file is part of "ACCESS IOT Stations".

"ACCESS IOT Stations" is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by the Free
Software Foundation, either version 3 of the License, or (at your option) any
later version.

"ACCESS IOT Stations" is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details.

You should have received a copy of the GNU General Public License along with
"ACCESS IOT Stations". If not, see <https://www.gnu.org/licenses/>.
'''

%matplotlib widget
from pymongo import MongoClient
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [2]:
# constant definitions
STATION = 'station1'
MONGO_IP = '10.224.83.51'
MONGO_PORT = 27017
DATABASE = 'stations'

# global variables
global_station = None
global_month = None  # holds a month of data
global_sensor = None

In [3]:
# connect to MongoDB
client = MongoClient(host=MONGO_IP, port=MONGO_PORT)
db = client[DATABASE]

In [4]:
# initialize all buttons and dropdowns

# buttons to select station
max_station = db['stations_info'].count_documents({'config': True})

stations_input = widgets.Dropdown(
    options=[(str(i), i) for i in range(max_station)],
    value=1,
    description='Number:',
)
stations_button = widgets.Button(
    description='Search Station',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Station',
)

# info / buttons to select month
month_text = widgets.Output()
month_text.append_stdout('Please select station before selecting month')

month_input = widgets.Dropdown(
    options=[(str(i), i) for i in range(1, 13)],
    value=1,
    description='Month:',
)
year_input = widgets.Dropdown(
    options=[(' ',' ')],
    value=' ',
    description='Year:',
)
month_button = widgets.Button(
    description='Search Month',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Month',
)

# buttons to select sensor
sensor_input = widgets.Dropdown(
    options=[(' ',' ')],
    value=' ',
    description='Sensor:',
)
sensor_button = widgets.Button(
    description='Search Sensor',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Sensor',
)

# buttons to select measurement
measurements_input = widgets.Dropdown(
    options=[(' ',' ')],
    value=' ',
    description='Measurements:',
)
measurements_button = widgets.Button(
    description='Graph',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Graph',
)

# graph to display later
plt.ioff()  # avoids showing the plotting widget right here and now.
plt.rcParams["figure.figsize"] = (10,10)

fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout = widgets.Layout( 
    max_width='1000px', 
    max_height='1000px',
    border='solid')

In [5]:
'''
Create all boxes to display
Boxes will be initially empty except for the one to select the station
As the user selects info, the next box should get filled and so on
'''

# b = Box(children=[m, fig.canvas], layout=box_layout)

# create box to house the station selection
station_box = widgets.Box(children=[stations_input, stations_button])

# All boxes below will appear empty until the button above selects something

# create box to house the month selection
month_box = widgets.Box(children=[month_input, year_input, month_button])

# create box to house the sensor selection
sensor_box = widgets.Box(children=[sensor_input, sensor_button])

# create box to house the measurement selection
measurements_box = widgets.Box(children=[measurements_input, measurements_button])

# graph box
graph_box = widgets.Box(children=[fig.canvas])

In [6]:
# display all boxes 

display(station_box)
display(month_text)
display(month_box)
display(sensor_box)
display(measurements_box)
display(graph_box)

Box(children=(Dropdown(description='Number:', index=1, options=(('0', 0), ('1', 1), ('2', 2)), value=1), Butto…

Output(outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 'Please select station before selecting mo…

Box(children=(Dropdown(description='Month:', options=(('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', …

Box(children=(Dropdown(description='Year:', options=((' ', ' '),), value=' '), Button(description='Search Sens…

Box(children=(Dropdown(description='Measurements:', options=((' ', ' '),), value=' '), Button(description='Gra…

Box(children=(Canvas(header_visible=False, layout=Layout(border_bottom='solid', border_left='solid', border_ri…

In [7]:
'''
What station to select -> what month to select -> what sensor to select -> what measurement to select
Create the functions for when a search button is pressed
When a search button is pressed, the server should pull up the relevant info for the next stage of the pipeline
When a button is pressed, all lower levels of the pipeline should be cleared
'''

##########
# Helper functions


##########
# Callback functions


@month_text.capture(clear_output=True, wait=True)
def stations_callback(btn: widgets.Button) -> None:
    '''
    Callback function for when user presses the search button for a station
    1. Checks the months the selected station has been collecting data for
    2. Update the month/year dropdowns
    3. Disable all search buttons below month
    4. Enable the month search button
    @param btn Search button for stations
    '''
    
    # collect value from dropdown
    station_to_search = f'station{stations_input.value}'
    global global_station
    global_station = station_to_search
    
    # search for data for next stage
    months = sorted(db[station_to_search].distinct('month'))
    
    min_month = months[0]
    max_month = months[-1]
    
    # update text to reflect month range
    #month_text.clear_output(wait=False)
    with month_text:
        print(f'Select a month between {min_month} and {max_month}')
    
    # update the possible dropdown values for the buttons at the month stage
    years = sorted(set([date.split('_')[0] for date in months]))  # all dates are format 'YYYY_MM'
    year_input.options = years
    year_input.value = years[0]
    
    # Enable the button
    month_button.disabled = False
    
    # disable all other buttons
    sensor_button.disabled = measurements_button.disabled = True


@month_text.capture()
def month_callback(btn: widgets.Button) -> None:
    '''
    Callback function for when user searches a specific month on a station
    1. Check if month is valid -> if not, end
    2. Collect all the sensors of that month
    3. Update dropdowns for sensors
    4. Enable the sensors button and disable the graph button
    '''
    
    # disable all buttons below month
    sensor_button.disabled = measurements_button.disabled = True
    
    # collect values from buttons
    month_to_search = f'{year_input.value}_{str(month_input.value).zfill(2)}'  # pads single-digits to have leading 0
    
    # check if month is valid

    if (data := db[global_station].find_one({'month': month_to_search})) is None:
        # update output
        month_text.append_stdout(f'Invalid output {month_to_search}\n')
        return
    
    month_text.append_stdout('Valid month, searching\n')
    
    # update values for sensor dropdown
    sensors = list(data.keys())
    sensors.remove('month')
    sensors.remove('_id')
    sensor_input.options = [(sen, sen) for sen in sensors]
    sensor_input.value = sensors[0]
    
    # enable button
    sensor_button.disabled = False
    
    # update global var
    global global_month
    global_month = data


@month_text.capture()
def sensor_callback(btn: widgets.Button) -> None:
    '''
    Callback function for when user searches a specific sensor on a station
    1. Check if month is valid -> if not, end
    2. Collect all the sensors of that month
    3. Update dropdowns for sensors
    4. Enable the sensors button and disable the graph button
    '''
    
    # disable all buttons below sensor
    measurements_button.disabled = True
    
    # collect values from buttons
    sensor = sensor_input.value
    global global_sensor
    global_sensor = sensor
    
    # update values for measurements dropdown
    measurements = list(global_month[sensor].keys())
    # clean list
    if 'type' in measurements:
        measurements.remove('type')
    if 'diagnostics' in measurements:
        measurements.remove('diagnostics')
    measurements_input.options = [(mes, mes) for mes in measurements]
    measurements_input.value = measurements[0]
    
    # enable button
    measurements_button.disabled = False


@month_text.capture()
def graph_callback(btn: widgets.Button) -> None:
    '''
    Callback function for when user searches a specific measurement on a sensor
    Graphs the chosen sensor
    '''
    
    fig.clear()
    
    # collect measurement
    measurement = measurements_input.value
    
    # collect x variable 
    x = global_month['gps']['datetime']
    
    y = global_month[global_sensor][measurement]
    
    if type(y) is list:
        plt.plot(x, y)
    elif type(y) is dict:
        '''
        If there are multiple sensors collecting that measurement, then the shape of the data should be:
        sensor: {
            measurement: {
                '0': <list_of_data>,
                '1': <list_of_data>,
                ...
            }
        }
        '''
        for data in y.values():
            plt.plot(x, data)
            
    fig.canvas.draw()
        
        


###########
# add all callbacks
stations_button.on_click(stations_callback)
month_button.on_click(month_callback)
sensor_button.on_click(sensor_callback)
measurements_button.on_click(graph_callback)
